# students-to-course-assigner

## Imports

In [ ]:
import os

import pandas as pd
import math

from dataclasses import dataclass, field
from typing import List

import random

from collections import defaultdict

## Read data

In [ ]:
excel_path = os.path.join("data", "InputNeufeld_Dez2023.xlsx")

In [ ]:
df_students = pd.read_excel(excel_path, sheet_name="Rohdaten").fillna("").reset_index()
df_courses = pd.read_excel(excel_path, sheet_name="Module")

In [ ]:
df_students.head()

In [ ]:
df_courses.head().fillna("")

In [ ]:
@dataclass
class Student:
    id_: int
    first_name: str
    last_name: str
    desired_courses: List[str]
    assigned_course: str = None

In [ ]:
s = Student(1, "j", "l", ["a1", "a2", "a3"])

In [ ]:
s

In [ ]:
student_per_student_id = {}
for e in df_students.to_dict(orient="records"):
    cs = []
    for i in range(1, 5):
        c = e[f"Def B - {i}. Prio"].replace(" ", "")
        cs += [c]
    if not all(not c for c in cs):
        if any(not c for c in cs):
            print(f"student has not all courses not defined, but any: {e}")
        else:
            id_ = e["index"]
            student_per_student_id[id_] = Student(id_, e["Vorname"], e["Name"], cs)
    else:
        print(f"student has all courses not defined: {e}")

In [ ]:
print(len(student_per_student_id))

In [ ]:
@dataclass
class Course:
    id_: str
    min_students: int
    max_students: int
    nb_wishes: int = 0
    assigned_students: List[int] = field(default_factory=lambda : [])
    def assign_if_possible(self, student):
        if len(self.assigned_students) < self.max_students:
            self.assigned_students += [student]
            student.assigned_course = self

In [ ]:
course_per_course_id = {}
for e in df_courses.to_dict(orient="records"):
    id_ = e["Module"]
    if id_.startswith("B"):
        course_per_course_id[id_] = Course(id_, e["Min"], e["Max"])

In [ ]:
print(len(course_per_course_id))

In [ ]:
nb_wishes_per_course_id = defaultdict(int)
for s in student_per_student_id.values():
    for c_id in s.desired_courses:
        nb_wishes_per_course_id[c_id] += 1
for c in course_per_course_id.values():
    c.nb_wishes = nb_wishes_per_course_id[c.id_]

In [ ]:
def init_assignment():
    for s in student_per_student_id.values():
        s.assigned_course = None
    for c in course_per_course_id.values():
        c.assigned_students = []

In [ ]:
def shuffle_list(ls):
    random.shuffle(ls)
    return ls

In [ ]:
init_assignment()
for s in shuffle_list(list(student_per_student_id.values())):
    n = len(s.desired_courses)
    a_desired_course = s.desired_courses[random.randint(0, n-1)]
    course = course_per_course_id[a_desired_course]
    course.assign_if_possible(s)

In [ ]:
students_without_course = [s for s in student_per_student_id.values() if s.assigned_course is None]
print(len(students_without_course))

In [ ]:
for c in course_per_course_id.values():
    print(c.id_, c.min_students, c.max_students, c.nb_wishes / 4, len(c.assigned_students), c.nb_wishes < c.min_students)